<a href="https://colab.research.google.com/github/darwinyusef/segmentacionYolo11yPython/blob/master/segmentacionDarwinYusefGonzalez14297510.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo Actividad 2 Evaluación de la segmentación

# PRESENTACIÓN
+ ### DARWIN YUSEF GONZALEZ TRIANA
+ ### C.C. 14297510
+ ### © Universidad Internacional de La Rioja (UNIR)
+ ### INVESTIGACIÓN EN IA


![Presentación](https://raw.githubusercontent.com/darwinyusef/20exHuggingFacePytorchTensorFlowSklearn/refs/heads/master/skitimage/intensity/presentacion.png)

## Objetivos

El objetivo de este trabajo es aprender a construir y evaluar el rendimiento de uno o más segmentadores. Esta actividad permitirá consolidar los conceptos sobre segmentación de imágenes aprendidos.

## Descripción

Este trabajo se entrega de forma individual. En él nos vamos a enfrentar a un verdadero problema de segmentación. La segmentación, como se ha visto, consiste en detectar regiones homogéneas y aislar/detectar objetos dentro de una imagen. Estas regiones habitualmente corresponden a los objetos que se están queriendo identificar.

Existen muchas maneras de enfocar este problema y puedes hacer uso de las técnicas de segmentación que consideres para resolverlo. Una vez elegidas estas técnicas, debes evaluar su rendimiento frente a imágenes de ground truth. En caso de que se utilicen partes de un software existente, deberá referenciarse la fuente. Debes mostrar en pantalla los resultados de los principales pasos.

In [ ]:
# lab ex 1
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from google.colab import files
# Cargar la imagen usando Pillow
from PIL import Image

# lab ex 2 clusters
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import linkage, fcluster
from sklearn.preprocessing import normalize
import time

In [ ]:
# cargar imagen
!wget https://raw.githubusercontent.com/darwinyusef/UsaHousingLab/refs/heads/master/image.png

# cargar roma
!wget https://raw.githubusercontent.com/darwinyusef/UsaHousingLab/refs/heads/master/roma.png

# cargar calle
!wget https://raw.githubusercontent.com/darwinyusef/UsaHousingLab/refs/heads/master/calle.png

# 1. Segmentación de colores sobre centroides basada en K-MEANS

Segmentación de colores sobre centroides basada en K-Means" es una técnica de procesamiento de imágenes que utiliza el algoritmo de clustering K-Means para agrupar los colores de los píxeles en una imagen. El proceso implica los siguientes pasos:

En esencia, esta técnica reduce la paleta de colores de una imagen agrupando colores similares y reemplazándolos por sus colores promedio (centroides). Es útil para simplificar imágenes, reducir la cantidad de colores necesarios para su representación y como un paso previo en algunas tareas de análisis de imágenes.

In [ ]:
image_path = "/content/image.png"

In [ ]:
print(f"Imagen cargada: {image_path}")

# Cargar la imagen usando Pillow
original_image = Image.open(image_path)
plt.imshow(original_image)
plt.title("Imagen Original")
plt.axis('off')
plt.show()

In [ ]:
# Convertir la imagen a un array numpy
image_array = np.array(original_image)

# Obtener las dimensiones de la imagen
height, width, channels = image_array.shape

# Reformatear la imagen para que cada píxel sea una fila
reshaped_image = image_array.reshape(height * width, channels)

print(f"Forma del array de la imagen original: {image_array.shape}")
print(f"Forma del array de la imagen reformateada: {reshaped_image.shape}")

In [ ]:
# @title Definir el número de clusters (segmentos)
n_clusters = 4 # Puedes experimentar con diferentes valores hasta un maximo de 8

In [ ]:
# @title Inicializar y entrenar el modelo K-means
kmeans = KMeans(n_clusters=n_clusters, random_state=0, n_init=10) # n_init para evitar problemas de convergencia
kmeans.fit(reshaped_image)

# Obtener las etiquetas de los clusters para cada píxel
cluster_labels = kmeans.predict(reshaped_image)

# Obtener los centros de los clusters (representan los colores promedio de cada segmento)
cluster_centers = kmeans.cluster_centers_

In [ ]:
# @title Inicia el entrenamiento no supervisado
import cv2

original_image = Image.open(image_path)
image_array = np.array(original_image)
height, width, channels = image_array.shape
reshaped_image = image_array.reshape(height * width, channels)

# Inicializar y entrenar el modelo K-means
kmeans = KMeans(n_clusters=n_clusters, random_state=0, n_init=10)
# aqui corremos el entrenados
kmeans.fit(reshaped_image)
# obtenemos las prediciones y el cluster_centers
cluster_labels = kmeans.predict(reshaped_image)
cluster_centers = kmeans.cluster_centers_

# Crear la imagen segmentada basada en los centros de los clusters
segmented_image_array = cluster_centers[cluster_labels].reshape(height, width, channels).astype(np.uint8)
# aparece la imagen segmentada (resultado)
segmented_image = Image.fromarray(segmented_image_array)



In [ ]:
# @title Grafica los segmentos y su respectivo histograma
# @markdown Es crucial entender que este codigo nos lleva a poder observar el cambio tan radical en comparación entre la imagen y el
# @markdown *la imagen segmentada*  nos permite comparar que efectivamente el listado de centroides de busqueda por k-means divide la imagen en colores especificos agrupando los mismos a traves del entrenamiento no supervisado

# ------------------- Histograma de la Imagen Original -------------------
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(original_image)
plt.title("Imagen Original")
plt.axis('off')

plt.subplot(1, 2, 2)
color = ('b', 'g', 'r') # El orden de los canales en OpenCV es BGR por defecto simplemente los cambio
# aplico la funcion del historigrama
for i, col in enumerate(color):
    histogram = cv2.calcHist([image_array], [i], None, [256], [0, 256])
    plt.plot(histogram, color=col)
    plt.xlim([0, 256])
plt.title("Histograma de Color (Original)")
plt.xlabel("Valor del Píxel")
plt.ylabel("Frecuencia")
plt.tight_layout()
plt.show()

# ------------------- Histograma de la Imagen Segmentada -------------------
segmented_image_array_cv2 = np.array(segmented_image) # Convertir a array para OpenCV
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(segmented_image)
plt.title(f"Imagen Segmentada con {n_clusters} Clusters")
plt.axis('off')

plt.subplot(1, 2, 2)
color = ('b', 'g', 'r')
for i, col in enumerate(color):
    histogram = cv2.calcHist([segmented_image_array_cv2], [i], None, [256], [0, 256])
    plt.plot(histogram, color=col)
    plt.xlim([0, 256])
plt.title("Histograma de Color (Segmentada)")
plt.xlabel("Valor del Píxel")
plt.ylabel("Frecuencia")
plt.tight_layout()
plt.show()

In [ ]:
# @title Exploración de la imagen creando una nueva a traves de reshape, sus cluster_labels, y sus clustes centros
# @markdown  De tal manera que obtenemos de los clusters los centros y sus respectivas agrupaciones ahora podemos verlo a traves de sus canales(channels) por medio de reshape

# Crear una nueva imagen basada en las etiquetas de los clusters y los centros
segmented_image_array = cluster_centers[cluster_labels].reshape(height, width, channels).astype(np.uint8)
print(channels)
# Convertir el array numpy a un objeto de imagen PIL
segmented_image = Image.fromarray(segmented_image_array)
fig, axes = plt.subplots(1, 2, figsize=(10, 5))



axes[0].imshow(original_image)
axes[0].set_title("Imagen Original")
axes[0].axis('off')

axes[1].imshow(segmented_image)
axes[1].set_title(f"Imagen Segmentada ({n_clusters} Clusters)")
axes[1].axis('off')

plt.tight_layout()
plt.show()

## A la hora de hacer la revisión de la segmentación por kmens se supone que esta debe reagrupar los elementos, aunque la segmentación contiee muchos puntos del mismo array se puede observar que los mismos se repiten mucho ejm 102/136/135 estos se repiten con base a la cantidad de clusters tiene la imagen

In [ ]:
# @markdown  [[[102 136 135]   [102 136 135]   [102 136 135]   ...
# print(segmented_image_array)

In [ ]:
# @title Suponiendo que 'original_image_array' es tu imagen original (antes de K-Means)

segmented_image_array = np.array(segmented_image)
num_pixels = segmented_image_array.shape[0] * segmented_image_array.shape[1]
# y 'labels_array' contiene la etiqueta del clúster asignada a cada píxel.

labels_array = np.random.randint(0, 3, size=(segmented_image_array.shape[0], segmented_image_array.shape[1])) # Ejemplo con 3 clústeres
num_clusters = np.max(labels_array) + 1

# Las dimensiones de 'labels_array' deberían ser (altura, ancho) de la imagen.
pixels = segmented_image_array.reshape(-1, 3) # Use the NumPy array here as well
flat_labels = labels_array.flatten()

In [ ]:
# @title Grafia de visualización 2D de numpy array [102 136 135]

plt.figure(figsize=(8, 6))
scatter = plt.scatter(pixels[:, 0], pixels[:, 1], c=flat_labels, cmap='viridis', s=5) # Usamos Rojo y Verde para el gráfico 2D

# Añadir etiquetas y título
plt.xlabel('Componente Rojo')
plt.ylabel('Componente Verde')
plt.title('Visualización de Clústeres K-Means (Proyección 2D)')

# Crear una leyenda para los clústeres
legend = plt.legend(*scatter.legend_elements(), title="Clústeres")
plt.gca().add_artist(legend)

plt.show()

## Es neceario observar que los clusters poseen una variación 3d ya que la posición z tambien se encuentra posicionada por esto la evaluación puede durar 40 seg

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

scatter = ax.scatter(pixels[:, 0], pixels[:, 1], pixels[:, 2], c=flat_labels, cmap='viridis', s=20)

ax.set_xlabel('Componente Rojo')
ax.set_ylabel('Componente Verde')
ax.set_zlabel('Componente Azul')
ax.set_title('Visualización de Clústeres K-Means (3D)')

legend = ax.legend(*scatter.legend_elements(), title="Clústeres")
ax.add_artist(legend)

plt.show()

## Explicación de la segmentación realizada a través del entrenamiento de modelo K-means

Este código realiza una segmentación de color de una imagen utilizando el algoritmo K-means. de mi parte realice toda la consuta ya que desde el inicio contamos con la respectiva segmentación

+ Carga y Preparación de la Imagen: La imagen original se abre y se convierte en un array NumPy. Luego, se reshapea para que cada fila represente un píxel y las columnas representen los canales de color (e.g., Rojo, Verde, Azul).

+ Inicialización y Entrenamiento de K-means: Se inicializa un modelo K-means con un número específico de clusters (n_clusters). Este modelo se entrena con los datos de los píxeles reshapeados para agrupar píxeles de colores similares en clusters. Se realizan múltiples inicializaciones (n_init=10) para obtener un mejor resultado.

+ Predicción de Clusters: Una vez entrenado, el modelo K-means predice a qué cluster pertenece cada píxel de la imagen. Estas etiquetas de cluster se almacenan en cluster_labels.

+ Definición de Paleta de Colores: Se define una lista de colores personalizados. El código verifica que haya suficientes colores definidos para el número de clusters especificado.

+ Creación de la Imagen Segmentada: Se crea una nueva imagen (array) con las mismas dimensiones que la original. A cada píxel de esta nueva imagen se le asigna el color correspondiente al cluster al que fue asignado por K-means, utilizando la paleta de colores definida.

+ Visualización: Finalmente, la imagen segmentada con los colores personalizados se convierte de nuevo en un objeto Image y se muestra utilizando matplotlib. El título indica el número de clusters utilizados.

In [ ]:
# @title la segmentación anterior la pintamos con un array de colores que permiten diferir la segmentación de las imagenes

original_image = Image.open(image_path)
image_array = np.array(original_image)
height, width, channels = image_array.shape
reshaped_image = image_array.reshape(height * width, channels)


# Inicializar y entrenar el modelo K-means
kmeans = KMeans(n_clusters=n_clusters, random_state=0, n_init=10)
kmeans.fit(reshaped_image)
cluster_labels = kmeans.predict(reshaped_image)

# Definir una paleta de colores para los clusters (ejemplo con algunos colores)
colors = [
    [255, 0, 0],
    [0, 255, 0],
    [0, 0, 255],
    [255, 255, 0],
    [255, 0, 255],
    [255, 100, 255],
    [100, 100, 0],
    [100, 50, 50]
]

# Asegurarse de que haya suficientes colores para todos los clusters
if n_clusters > len(colors):
    raise ValueError(f"Se necesitan al menos {n_clusters} colores en la paleta.")

# Crear la imagen segmentada con colores personalizados
segmented_image_array_rainbow = np.zeros((height * width, channels), dtype=np.uint8)
for i in range(n_clusters):
    segmented_image_array_rainbow[cluster_labels == i] = colors[i]

segmented_image_array_rainbow = segmented_image_array_rainbow.reshape(height, width, channels)
segmented_image_rainbow = Image.fromarray(segmented_image_array_rainbow)

# Mostrar la imagen segmentada con colores personalizados
plt.imshow(segmented_image_rainbow)
plt.title(f"Imagen Segmentada con {n_clusters} Clusters con visualización variada por color")
plt.axis('off')
plt.show()

Se podría usar para extraer las paletas de colores dominantes de imágenes o diseños. Creación de Muestrarios de Color: Agrupar colores similares puede ayudar a crear los siguientes elementos a continuación.

1. Edición y Mejora de Imágenes:
2. Análisis de Imágenes y Visión por Computadora:
3. Aplicaciones Médicas:
4. Teledetección y Geografía:
5. Control de Calidad Industrial:
6. Marketing y Diseño:



# 2. Exploración de la Umbralización para segmentación a traves del uso de

+ THRESH_BINARY
+ THRESH_BINARY_INV
+ THRESH_BINARY
+ THRESH_OTSU
+ THRESH_BINARY
+ THRESH_BINARY

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

In [ ]:
image_path = '/content/roma.png'
img = cv2.imread(image_path)

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

fig, axes = plt.subplots(1, 2, figsize=(10, 5))

axes[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)) # Display in RGB for matplotlib
axes[0].set_title("Imagen Original")
axes[0].axis('off')

axes[1].imshow(gray, cmap='gray') # Specify grayscale colormap
axes[1].set_title("Imagen en Escala de Grises")
axes[1].axis('off')

plt.tight_layout()
plt.show()

## Aplicar diferentes técnicas de umbralización

La umbralización es un proceso para segmentar una imagen, separando objetos de interés del fondo basándose en la intensidad de los píxeles. El código aplica varias técnicas:
Umbralización binaria simple (cv2.THRESH_BINARY y cv2.THRESH_BINARY_INV): Establece los píxeles a un valor blanco o negro dependiendo de si su intensidad está por encima o por debajo de un umbral fijo (thresh_value). La versión INV invierte esta asignación.
Umbralización de Otsu (cv2.THRESH_BINARY + cv2.THRESH_OTSU): Un método automático para encontrar un valor de umbral óptimo basado en el histograma de la imagen.
Umbralización adaptativa (cv2.ADAPTIVE_THRESH_MEAN_C y cv2.ADAPTIVE_THRESH_GAUSSIAN_C): En lugar de un umbral global, calcula un umbral local para cada píxel basándose en la intensidad de los píxeles vecinos. Los métodos MEAN_C y GAUSSIAN_C utilizan diferentes formas de calcular este umbral local (la media o una media ponderada por una función gaussiana).

In [ ]:
# @title Explorando una gama basica de Umbralización binaria para darle uso a procesos de segmentación


image_path = '/content/roma.png'
img = cv2.imread(image_path)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Umbralización binaria simple
thresh_value = 128
_, thresh_binary = cv2.threshold(gray, thresh_value, 255, cv2.THRESH_BINARY)
_, thresh_binary_inv = cv2.threshold(gray, thresh_value, 255, cv2.THRESH_BINARY_INV)
_, thresh_otsu = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
thresh_adapt_mean = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
thresh_adapt_gauss = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

fig, axes = plt.subplots(2, 3, figsize=(12, 8))

# Mostrar imagen original
axes[0, 0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title("Imagen Original")
axes[0, 0].axis('off')

# Mostrar imagen en escala de grises
axes[0, 1].imshow(gray, cmap='gray')
axes[0, 1].set_title("Imagen en Escala de Grises")
axes[0, 1].axis('off')

# Mostrar umbralización binaria simple
axes[0, 2].imshow(thresh_binary, cmap='gray')
axes[0, 2].set_title(f"Umbral Binario ({thresh_value})")
axes[0, 2].axis('off')

# Mostrar umbralización binaria inversa
axes[1, 0].imshow(thresh_binary_inv, cmap='gray')
axes[1, 0].set_title(f"Umbral Binario Inverso ({thresh_value})")
axes[1, 0].axis('off')

# Mostrar umbralización de Otsu
axes[1, 1].imshow(thresh_adapt_gauss, cmap='gray')
axes[1, 1].set_title(f"Umbral de Otsu")
axes[1, 1].axis('off')

# Mostrar umbralización adaptativa (Media)
axes[1, 2].imshow(thresh_adapt_mean, cmap='gray')
axes[1, 2].set_title("Umbral Adaptativo (Media)")
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()


# 3. Segmentación de imagenes a traves de selección gama de Umbralización  y / o agrupamiento de contornos
###### (TENER EN CUENTA EL TEMA ANTERIOR)
Este código realiza una segmentación de imagen utilizando el algoritmo SLIC (Simple Linear Iterative Clustering). Primero, convierte la imagen a formato RGB y luego a valores flotantes. SLIC agrupa píxeles similares en superpíxeles compactos. Las regiones segmentadas se pintan con colores promedio, y luego se dibujan bordes azules para visualizarlos claramente. El resultado final es una imagen donde los objetos o regiones con características similares están delimitados por bordes de color. Esto es útil para simplificar el análisis de imágenes, identificar objetos y reducir la complejidad para tareas posteriores como el reconocimiento.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.segmentation import slic, mark_boundaries
from skimage.util import img_as_float
from skimage.segmentation import slic, mark_boundaries
from skimage.color import label2rgb
from skimage.util import img_as_float
from google.colab.patches import cv2_imshow

In [ ]:
# @title Encontrar contornos, aqui solo usamos TRESH_BINARY PERO COMO SE OBSERVO ANTES SE PUEDE AMPLIAR MUCHO MÁS
_, thresh = cv2.threshold(gray, thresh_value, 255, cv2.THRESH_BINARY) # <--- SOLO DEBES CAMBIAR EL TIPO DE UMBRALIZACIÓN PARA SELECCIÓN DE CONTORNOS
# cv2_imshow(thresh, f"Umbralización Binaria (Umbral = {thresh_value})") #This line is changed
cv2_imshow(thresh) #updated call to cv2_imshow
print(f"Umbralización Binaria (Umbral = {thresh_value})")
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

print(f"Se encontraron {len(contours)} contornos.")

In [ ]:
# @title Segmentación de contornos usando SIC

# Cargar imagen
image_path = '/content/roma.png'
image = cv2.imread(image_path)
image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
image_float = img_as_float(image)

# @markdown Aplicamos SLIC a traves de super pixels
segments = slic(image_float, n_segments=250, compactness=10, start_label=1)

# @markdown Dibujar contornos con mark_boundaries
output = mark_boundaries(image, segments, color=(0, 0, 0))
img_contours = img.copy()
# @title Dibujar los contornos
cv2.drawContours(img_contours, contours, -1, (0, 255, 0), 2) # Color verde, grosor 2

cv2_imshow(img_contours)


### COMO SE OBSERVA EN ESTE CUERPO SE GENERA OTRO EJEMPLO QUE PERMITE OBSERVAR LA MISMA TECNICA PERO AHORA DANDO USO A LOS SUPERPIXELS

In [ ]:
# @title Mostrar resultado de segmentación tipo SLIC SUPERPIXEL para muestra de contornos por agrupamiento de la imagen
plt.figure(figsize=(8, 8))
plt.title('Segmentación SLIC')
plt.imshow(output)
plt.axis('off')
plt.show()

Este código carga una imagen y aplica el algoritmo SLIC para segmentarla en superpíxeles. Primero, convierte la imagen a RGB y luego a valores flotantes. SLIC agrupa píxeles similares en regiones compactas. Luego, mark_boundaries dibuja los bordes de estos superpíxeles en negro sobre la imagen original. Finalmente, intenta dibujar contornos detectados (aunque la detección real de contornos no está explícitamente en este fragmento) en verde sobre una copia de la imagen original, mostrando el resultado con cv2_imshow. El objetivo es visualizar la segmentación de la imagen mediante contornos.

In [ ]:
# @title uso de mascaras de tipo SUPERPIXEL para reconocimiento colormetrico de objetos

image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image_float = img_as_float(image)

# Aplicar SLIC
segments = slic(image_float, n_segments=250, compactness=10, start_label=1)

# Pintar regiones con colores aleatorios
image_segments = label2rgb(segments, image, kind='avg')

# @markdown Dibujar bordes sobre la imagen segmentada <-- COMO SE OBSERVA EN EL ANTERIOR mark_boundaries permite la especificación de contornos asimismo permitiendo la colorización
output = mark_boundaries(image_segments, segments, color=(0, 0, 1))  # Borde azul

# Mostrar resultado final
plt.figure(figsize=(10, 10))
plt.title('Segmentación SUPERPIXEL tipo máscara con bordes')
plt.imshow(output)
plt.axis('off')
plt.show()

# 4. Segmentación de imagen para obtención del fondo usando tecnias de color LAB y rembg

Este código Python primero elimina el fondo de una imagen usando la librería rembg. Luego, convierte la imagen resultante a un formato manejable por OpenCV y extrae su canal alfa (transparencia) para crear una máscara del objeto.
A continuación, aplica el algoritmo de clustering K-means sobre la imagen convertida al espacio de color LAB para segmentar los colores en dos grupos, identificando el fondo por su frecuencia. Se crea otra máscara basada en esta segmentación.
Finalmente, combina ambas máscaras para aislar el objeto principal (sin fondo) que también pertenece al grupo de color predominante del objeto, mostrando los resultados.

In [ ]:
!pip install rembg
!pip install onnxruntime

In [ ]:
# Aqui se presenta como la gestión de color usando los TERM CRITERIA Y color labs se pueden crear mascaras de condo para asi retirarlo
image_path = '/content/calle.png'
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Redimensionar (opcional)
scale_percent = 50  # Reducir tamaño
width = int(image.shape[1] * scale_percent / 100)
height = int(image.shape[0] * scale_percent / 100)
image = cv2.resize(image, (width, height))

# Convertir imagen a 2D para clustering
pixel_values = image.reshape((-1, 3))
pixel_values = np.float32(pixel_values)

# Definir K-means
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
k = 2  # Fondo vs Objeto
_, labels, centers = cv2.kmeans(pixel_values, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

# Convertimos a uint8
centers = np.uint8(centers)
segmented_image = centers[labels.flatten()]
segmented_image = segmented_image.reshape(image.shape)

# Crear máscara para retirar fondo
labels = labels.flatten()
background_label = np.argmax(np.bincount(labels))  # El cluster más grande es el fondo
mask = np.where(labels == background_label, 0, 1)
mask = mask.reshape((image.shape[0], image.shape[1]))

# Aplicar máscara
result = image * mask[:, :, np.newaxis]

# Mostrar resultados
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.title('Imagen Original')
plt.imshow(image)
plt.axis('off')

plt.subplot(1, 3, 2)
plt.title('Segmentación')
plt.imshow(segmented_image)
plt.axis('off')

plt.subplot(1, 3, 3)
plt.title('Objeto de fondo invertido')
plt.imshow(result)
plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
from rembg import remove
import cv2
import numpy as np
from PIL import Image
import io
import matplotlib.pyplot as plt

# @markdown Leer imagen original como bytes
input_path = '/content/calle.png'
with open(input_path, 'rb') as f:
    input_image = f.read()

# @markdown Elimina el fondo con rembg y convertir a PIL Image con transparencia (RGBA)
output_image = remove(input_image)
output_pil = Image.open(io.BytesIO(output_image)).convert('RGBA')

# @markdown Convertierte PIL Image a array numpy y separar RGB y canal alfa
image_rgba = np.array(output_pil)
image_rgb = cv2.cvtColor(image_rgba[:, :, :3], cv2.COLOR_RGB2BGR)
alpha = image_rgba[:, :, 3]

# @markdown Crear máscara binaria donde el alfa > 0 es 1 (objeto) y 0 (fondo)
mask_rembg = np.where(alpha > 0, 1, 0).astype(np.uint8)

# @markdown Convertir imagen RGB a espacio de color LAB para mejor segmentación por color
lab_image = cv2.cvtColor(image_rgb, cv2.COLOR_BGR2LAB)
pixel_values = lab_image.reshape((-1, 3)).astype(np.float32)

#  @markdown Aplicar K-means clustering para segmentar colores en 2 grupos
k = 2
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
_, labels, centers = cv2.kmeans(pixel_values, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

labels = labels.flatten()
#  @markdown Identificar la etiqueta del fondo encontrando el color más frecuente
background_label = np.argmax(np.bincount(labels))

# @markdown Crear máscara de segmentación donde la etiqueta del fondo es 0 y otras son 1
mask_segment = np.where(labels == background_label, 0, 1).astype(np.uint8)
mask_segment = mask_segment.reshape((image_rgb.shape[0], image_rgb.shape[1]))

# @markdown Combinar las dos máscaras (rembg y segmentación): solo el objeto sin fondo Y segmentado
final_mask = cv2.bitwise_and(mask_rembg, mask_segment)

# @markdown Aplicar la máscara final a la imagen original para extraer el objeto segmentado
result = cv2.bitwise_and(image_rgb, image_rgb, mask=final_mask)

# Mostrar los resultados: imagen rembg, máscara de segmentación y objeto final
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1), plt.imshow(image_rgba), plt.title('Rembg'), plt.axis('off')
plt.subplot(1, 3, 2), plt.imshow(mask_segment, cmap='gray'), plt.title('Segmentación LAB + KMeans'), plt.axis('off')
plt.subplot(1, 3, 3), plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB)), plt.title('Objeto Final'), plt.axis('off')
plt.tight_layout(), plt.show()

# 5. Pruebas de Detección de Anomalías y Segmentación Binaria utilizando el dataframe a traves del entrenamiento,  uso de tensorflow y su rendimiento frente a imágenes de ground truth. https://github.com/BIDS/BSDS500 y el AUTOENCODER CONVOLUCIONAL

Este código implementa un autoencoder convolucional para la detección de anomalías en imágenes. Primero, load_images carga y preprocesa imágenes de un directorio. Luego, se define un autoencoder con una fase de codificación (Conv2D y MaxPooling) para reducir la dimensionalidad y una fase de decodificación (Conv2D y UpSampling2D) para reconstruir la imagen. El modelo se entrena para minimizar el error de reconstrucción en datos normales. Finalmente, detect_anomaly compara la imagen original con su reconstrucción para generar un mapa de anomalías y una máscara binaria, resaltando las regiones con mayores diferencias como posibles anomalías.

Después de que el autoencoder reconstruye la imagen de entrada, se calcula la diferencia absoluta pixel a pixel entre la imagen original y la reconstruida. Luego, se crea un mapa de anomalías tomando la media de esta diferencia a través de los canales de color (RGB).

La segmentación binaria se logra al aplicar un umbral (en este caso, 0.1) al mapa de anomalías. Cualquier valor en el mapa de anomalías que sea mayor que este umbral se considera parte de una posible anomalía y se establece en 1 (blanco) en la máscara de anomalías. Los valores por debajo del umbral se establecen en 0 (negro).

El resultado es una máscara binaria donde las áreas blancas resaltan las regiones de la imagen original que el autoencoder no pudo reconstruir con precisión, lo que sugiere la presencia de anomalías.

# Que es entonces el autoencoder convolucoinal

Después de que el autoencoder reconstruye la imagen de entrada, se calcula la diferencia absoluta pixel a pixel entre la imagen original y la reconstruida. Luego, se crea un mapa de anomalías tomando la media de esta diferencia a través de los canales de color (RGB).

La segmentación binaria se logra al aplicar un umbral (en este caso, 0.1) al mapa de anomalías. Cualquier valor en el mapa de anomalías que sea mayor que este umbral se considera parte de una posible anomalía y se establece en 1 (blanco) en la máscara de anomalías. Los valores por debajo del umbral se establecen en 0 (negro).

El resultado es una máscara binaria donde las áreas blancas resaltan las regiones de la imagen original que el autoencoder no pudo reconstruir con precisión, lo que sugiere la presencia de anomalías.

esta información se obtuvo de un curso que hice de UDEMY https://www.udemy.com/course/modern-computer-vision

La verdad queria probar hace rato la ejecución de Autoencoders y pues gracias a este tutorial pude lograr identificar que estos tambien sirven para ralización de segmentación

In [ ]:
!pip install opencv-contrib-python --upgrade

In [ ]:
!git clone https://github.com/BIDS/BSDS500.git

In [ ]:
DATA_PATH = "/content/BSDS500/BSDS500/data/images/"

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.optimizers import Adam

In [ ]:
# Paso 3: Cargar imágenes y preparar datos
def load_images(folder, size=(128, 128)):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, size)
            img = img / 255.0
            images.append(img)
    return np.array(images)

images = load_images(DATA_PATH + 'train')
print("Total imágenes:", images.shape)

# Paso 4: Crear Autoencoder para detección de anomalías
input_img = Input(shape=(128, 128, 3))

# Encoder
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# Decoder
x = Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer=Adam(1e-3), loss='mse')
autoencoder.summary()

# Paso 5: Entrenar Autoencoder
x_train, x_test = train_test_split(images, test_size=0.2, random_state=42)
autoencoder.fit(x_train, x_train, epochs=30, batch_size=32, shuffle=True, validation_data=(x_test, x_test))

# Paso 6: Detección de Anomalías y Segmentación Binaria
def detect_anomaly(img):
    pred = autoencoder.predict(img[np.newaxis, ...])[0]
    diff = np.abs(img - pred)
    anomaly_map = np.mean(diff, axis=2)  # Mapa de Anomalía
    anomaly_mask = (anomaly_map > 0.1).astype(np.uint8)  # Segmentación Binaria
    return pred, anomaly_map, anomaly_mask

In [ ]:
# Paso 7: Pruebas
idx = np.random.randint(0, len(x_test))
img = x_test[idx]

deconstruida, mapa_anomalia, mascara = detect_anomaly(img)

plt.figure(figsize=(12, 6))
plt.subplot(1, 4, 1)
plt.title("Imagen Original")
plt.imshow(img)

plt.subplot(1, 4, 2)
plt.title("Deconstruccion")
plt.imshow(deconstruida)

plt.subplot(1, 4, 3)
plt.title("Mapa Anomalía")
plt.imshow(mapa_anomalia, cmap='hot')

plt.subplot(1, 4, 4)
plt.title("Segmentación")
plt.imshow(mascara, cmap='gray')

plt.show()

Total params: 13,347
Trainable params: 13,347
Non-trainable params: 0

La evaluación de performance en la detección de anomalías mediante un autoencoder se centra en cuantificar qué tan bien el modelo identifica y localiza las irregularidades en las imágenes. El *ground truth* o verdad fundamental son las anotaciones o etiquetas que indican la ubicación y extensión real de las anomalías en un conjunto de imágenes de prueba.

Para evaluar el rendimiento, se comparan las máscaras de anomalía generadas por el autoencoder con las máscaras de *ground truth*. Métricas comunes incluyen la precisión (precision), el recall (exhaustividad), la puntuación F1 (equilibrio entre precisión y recall) y el IoU (Intersection over Union) para la segmentación.

El código proporcionado implementa un autoencoder convolucional para la detección de anomalías. Los Pasos 3, 4 y 5 preparan los datos, definen la arquitectura del autoencoder y lo entrenan para reconstruir imágenes normales. El Paso 6 genera un mapa de anomalía basado en la diferencia entre la imagen original y su reconstrucción, y luego aplica un umbral para crear una máscara binaria de segmentación.

Para evaluar el rendimiento frente al ground truth, se necesitarían cargar las imágenes de ground truth correspondientes a las imágenes de prueba. Luego, para cada imagen de prueba, se aplicaría la función detect_anomaly y se compararían la máscara de anomalía resultante (anomaly_mask) con su máscara de ground truth. Se calcularían las métricas mencionadas anteriormente sobre el conjunto de imágenes de prueba para obtener una evaluación global del rendimiento del modelo.

# BIBLIOGRAFÍA

+ Burney, S. M. A., & Tariq, H. (2014). K-Means Cluster Analysis for Image Segmentation. International Journal of Computer Applications, 96(4), 1–8. https://doi.org/10.5120/16779-6360

+ Sammouda, R., & El-Zaart, A. (2021). An Optimized Approach for Prostate Image Segmentation Using K-Means Clustering Algorithm with Elbow Method. Computational Intelligence and Neuroscience, 2021, 1–10. https://doi.org/10.1155/2021/4553832

+ Datanovia. (n.d.). Determining the optimal number of clusters: 3 must-know methods. Retrieved from https://www.datanovia.com/en/lessons/determining-the-optimal-number-of-clusters-3-must-know-methods/

+ Winland, V. (n.d.). K-means clustering: Definition, applications, use cases. IBM. Retrieved from https://www.ibm.com/think/topics/k-means-clustering

+ Chauhan, N. S. (2019, August 9). Introduction to Image Segmentation with K-Means clustering. KDnuggets. Retrieved from https://www.kdnuggets.com/2019/08/introduction-image-segmentation-k-means-clustering.html

+ Tomar, A. (2025, March 13). Elbow Method in K-Means Clustering: Definition and Steps. Built In. Retrieved from https://builtin.com/data-science/elbow-method

+ Hamza, M. (2024, November 4). K-Means Clustering for Image Segmentation Using OpenCV in Python. Towards Singularity. Retrieved from https://medium.com/towardssingularity/k-means-clustering-for-image-segmentation-using-opencv-in-python-17178ce3d6f3

+ Safak, A. N. (2024, December 18). How to Interpret Silhouette Plot for K-Means Clustering. Medium. Retrieved from https://medium.com/@favourphilic/how-to-interpret-silhouette-plot-for-k-means-clustering-414e144a17fe

+ Mahmood, M. (2020, March 16). Text Clustering: Comparing TF-IDF, BERT, and SBERT Embeddings with K-Means Clustering. Python in Plain English. Retrieved from https://python.plainenglish.io/text-clustering-comparing-tf-idf-bert-and-sbert-embeddings-with-k-means-clustering-a965fe7a69cc

+ First Principles of Computer Vision. (n.d.). K-Means Segmentation | Image Segmentation. YouTube.(https://www.youtube.com/watch?v=BGBrK0d_nug)

+ First Principles of Computer Vision. (n.d.). k-Means Segmentation | Image Segmentation. YouTube. https://www.youtube.com/watch?v=10cD46dQlgU

+ Simplilearn. (2015). The Elbow Method for K-Means Clustering. YouTube. https://www.youtube.com/watch?v=ht7geyMAFfA

+ Simplilearn. (2014). K-Means Clustering. YouTube. https://www.youtube.com/watch?v=22mpExWh1LY

+ iddiqui, F. U., & Yahya, A. (2022). Clustering Techniques for Image Segmentation. Springer.

+ Shovon, M. H. I. (2012). Clustering and Image Segmentation: Determining the number of cluster and Color Image Segmentation using K means. LAP LAMBERT Academic Publishing.
